# <center style="font-size:140%;"> Report - Problem Set #*2*
 <center> Group Members:  Giacomo Martiriggiano, Mattia Gallese, Sophie De Becker, Yao Di

This Jupyter Notebook will outline the data cleaning process and binary classification for the churn factor of customers.csv for the assignment

## Data loading 

In [1]:
#import modules needed for data analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn import svm
#read the csv file to transfer all the data into "data"
data = pd.read_csv('customers.csv')

## Data Preprocessing
We keep the data cleaning process as what we did for the Problem Set 1

In [2]:
data['TotalCharges'] = data['TotalCharges'].replace(" ", np.nan).astype('float32') 
data["SeniorCitizen"]=data["SeniorCitizen"].astype("object")
data = data[data["TotalCharges"].notnull()]
data = data.reset_index()[data.columns]
#now we forced the 0 and 1 to be objects and we know we can drop the empty values in Total charger 

Finally a number may be non-empty but still unrealistic for example negative tenure. Let's now check that tenure and charges are non negative

In [3]:
print((data['tenure'] <0).any())
print((data['TotalCharges']<0).any())
print((data['MonthlyCharges']<0).any())

False
False
False


We also need to encode the inputs from enumerates to numbers

In [4]:
data_E2N = data[['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod','Churn']].copy()
data_E2N = data_E2N.apply(lambda x: pd.factorize(x)[0])
scaler = MinMaxScaler()
data_E2N = pd.DataFrame(scaler.fit_transform(data_E2N),columns=data_E2N.columns)
data_N = data[['tenure', 'MonthlyCharges', 'TotalCharges']].copy()
data_N['TotalCharges'] = pd.to_numeric(data_N['TotalCharges'])
scaler2 = StandardScaler()
data_N = pd.DataFrame(scaler2.fit_transform(data_N),columns=data_N.columns)

In [5]:
data_numerized = pd.concat([data_E2N, data_N], axis=1)
scaler = MinMaxScaler()
data_final = pd.DataFrame(scaler.fit_transform(data_numerized),columns=data_numerized.columns)

## Problems Defination
The key of the problem set is to develop a method to predict the performance of the customer. The prediction problem could be easily transfered to asup

The key output of our supervised learning model is the "Churn" parameter. The first step is to explore it.

In [6]:
data_final.Churn.value_counts()

0.0    5163
1.0    1869
Name: Churn, dtype: int64

As we could observe, the dataset is not balanced based on the churn parameter. In order to get a reliable result, we need to downsampling the No Churn set of customer. 

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data_final, data_final["Churn"], stratify=data_final["Churn"],test_size=0.3)
count_nochurn, count_churn = x_train["Churn"].value_counts()

x_train_nochurn = x_train[x_train['Churn'] == 0.0]
x_train_churn = x_train[x_train['Churn'] == 1.0]

x_train_nochurn_resample = x_train_nochurn.sample(count_churn)
x_train_resample = pd.concat([x_train_nochurn_resample, x_train_churn], axis=0)

## Problem Solving

For the binary classification problem with multiple dimensions, support vector machine is a common solution to find a robust classifier

In [11]:
classifier = svm.SVC(gamma='auto')
classifier.fit(x_train_resample.drop(columns="Churn"), x_train_resample["Churn"])
y_predict=classifier.predict(x_test.drop(columns="Churn"))

In [12]:
print("The accuracy score for the classifier is :",accuracy_score(y_test,y_predict))
print("The recall score for the classifier is :",recall_score(y_test,y_predict,labels=[1,0]))
print("The precision score for the classifier is ::",precision_score(y_test,y_predict,labels=[1,0]))
pd.DataFrame(confusion_matrix(y_test,y_predict,labels=[1,0]), ["Churn_true","No Churn_true"], ["Churn_pred","No Churn_pred"])

The accuracy score for the classifier is : 0.718957345971564
The recall score for the classifier is : 0.8163992869875223
The precision score for the classifier is :: 0.4831223628691983


,Churn_pred,No Churn_pred
Churn_true,458,103
No Churn_true,490,1059


We could add cross validation to avoid overfitting

In [13]:
Accuracy_scores = []
Recall_scores = []
Precision_scores = []

k = 4
classifier = svm.SVC(gamma='auto')
cv = KFold(n_splits=k, random_state=42, shuffle=False)
for train_index, test_index in cv.split(data_final):
    x_train, x_test, y_train, y_test = data_final.iloc[train_index], data_final.iloc[test_index], data_final.Churn.iloc[train_index], data_final.Churn.iloc[test_index]
    
    x_train_nochurn = x_train[x_train['Churn'] == 0.0]
    x_train_churn = x_train[x_train['Churn'] == 1.0]

    x_train_nochurn_resample = x_train_nochurn.sample(count_churn)
    x_train_resample = pd.concat([x_train_nochurn_resample, x_train_churn], axis=0)
    
    classifier.fit(x_train_resample.drop(columns="Churn"), x_train_resample["Churn"])
    y_predict=classifier.predict(x_test.drop(columns="Churn"))
    Accuracy_scores.append(accuracy_score(y_test,y_predict))
    Recall_scores.append(recall_score(y_test,y_predict,labels=[1,0]))
    Precision_scores.append(precision_score(y_test,y_predict,labels=[1,0]))
    print(pd.DataFrame(confusion_matrix(y_test,y_predict,labels=[1,0]), ["Churn_true","No Churn_true"], ["Churn_pred","No Churn_pred"]))

print("The accuracy scores for the ",k, "-fold classifier is :",Accuracy_scores)
print("The recall scores for the ",k, "-fold classifier is :",Recall_scores)
print("The precision scores for the ",k, "-fold classifier is :",Precision_scores)

               Churn_pred  No Churn_pred
Churn_true            398             62
No Churn_true         438            860
               Churn_pred  No Churn_pred
Churn_true            390             67
No Churn_true         402            899
               Churn_pred  No Churn_pred
Churn_true            374             92
No Churn_true         477            815
               Churn_pred  No Churn_pred
Churn_true            398             88
No Churn_true         418            854
The accuracy scores for the  4 -fold classifier is : [0.7155858930602957, 0.7332195676905574, 0.6763367463026166, 0.7121729237770194]
The recall scores for the  4 -fold classifier is : [0.8652173913043478, 0.8533916849015317, 0.8025751072961373, 0.8189300411522634]
The precision scores for the  4 -fold classifier is : [0.47607655502392343, 0.49242424242424243, 0.4394829612220917, 0.4877450980392157]
